This is a notebook that merges the parquet files with four datasets taken from https://dbr.abs.gov.au/region.html?lyr=ra&rgn=20 which are for major cities, inner regional, outer regional and remote australia inside victoria


In [3]:
# downloading datasets
import urllib.request as req

# url = [major city, inner regional, outer regional, remote]
url = ['https://dbr.abs.gov.au/processedData/csv/RA_20.csv', 'https://dbr.abs.gov.au/processedData/csv/RA_21.csv','https://dbr.abs.gov.au/processedData/csv/RA_22.csv','https://dbr.abs.gov.au/processedData/csv/RA_23.csv']
url_header = ['major', 'inner_r', 'outer_r', 'remote']
urls = {}
for i,j in enumerate(url_header):
    req.urlretrieve(url[i], '../data/raw/{}.csv'.format(j))

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/abs/major.csv'

In [14]:
# converting csv files to spark dataframe
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("CSV to DataFrame").getOrCreate()


In [17]:
dfs = []
url_header = ['major', 'inner_r', 'outer_r', 'remote']
region = ['metropolitan victoria', 'inner regional victoria', 'outer regional victoria', 'remote victoria']

i = 1
while i < 5:
    df = spark.read.csv('../data/raw/{}.csv'.format(url_header[i-1]), header=True, inferSchema=True)
    
    # only using columns from 2018-2022
    df = df.drop('2011', '2015', '2016', '2023')
    # assigning region values
    df = df.withColumn('region', F.lit(region[i-1]))
    dfs.append(df)
    i += 1

In [22]:
from functools import reduce

merged_df = reduce(lambda df1, df2: df1.union(df2), dfs)

In [25]:
merged_df.write.parquet('../data/raw/region_summary.parquet')